In [1]:
# installing kaggle library
!pip install kaggle

In [2]:
#importing needed libraries
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix

In [3]:
#import your file kaggle.json
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp /kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


**API to fetch dataset from kaggle**




In [4]:
!kaggle datasets download -d dhoogla/cicddos2019 #downloading our dataset in zip format

 59% 17.0M/28.7M [00:00<00:00, 45.6MB/s]
100% 28.7M/28.7M [00:00<00:00, 62.9MB/s]


In [5]:
from zipfile import ZipFile #extacting our zip file
datasetddos = '/content/cicddos2019.zip'
with ZipFile(datasetddos,'r') as zip :
  zip.extractall()
  print("the dataset is extracted")

the dataset is extracted


In [6]:
#trying to gather all in one parquet file
#1-putting every parquet file into a dataframe
#2-making a list which will hold our dataframes
#3-concatinate the list using pandas function concat()
#4-converting the concatinated dataframe to parquet file .
df1 = pd.read_parquet('/content/DNS-testing.parquet')
df2 = pd.read_parquet('/content/LDAP-testing.parquet')
df3 = pd.read_parquet('/content/LDAP-training.parquet')
df4 = pd.read_parquet('/content/MSSQL-testing.parquet')
df5 = pd.read_parquet('/content/MSSQL-training.parquet')
df6 = pd.read_parquet('/content/NTP-testing.parquet')
df7 = pd.read_parquet('/content/NetBIOS-testing.parquet')
df8 = pd.read_parquet('/content/NetBIOS-training.parquet')
df9 = pd.read_parquet('/content/Portmap-training.parquet')
df10 = pd.read_parquet('/content/SNMP-testing.parquet')
df11 = pd.read_parquet('/content/Syn-testing.parquet')
df12 = pd.read_parquet('/content/Syn-training.parquet')
df13 = pd.read_parquet('/content/TFTP-testing.parquet')
df14 = pd.read_parquet('/content/UDP-testing.parquet')
df15 = pd.read_parquet('/content/UDP-training.parquet')
df16 = pd.read_parquet('/content/UDPLag-testing.parquet')
df17 = pd.read_parquet('/content/UDPLag-training.parquet')
dfs = []
dfs.append(df1)
dfs.append(df2)
dfs.append(df3)
dfs.append(df4)
dfs.append(df5)
dfs.append(df6)
dfs.append(df7)
dfs.append(df8)
dfs.append(df9)
dfs.append(df10)
dfs.append(df11)
dfs.append(df12)
dfs.append(df13)
dfs.append(df14)
dfs.append(df15)
dfs.append(df16)
dfs.append(df17)
concatenated_df = pd.concat(dfs, ignore_index=True)
output_file = "/content/finaldataset.parquet"
concatenated_df.to_parquet(output_file, engine='pyarrow')





In [7]:
dfp = pd.read_parquet('/content/finaldataset.parquet') #putting all parquet files into one dataframe
dfp.head()
dfp['Label'].value_counts()

DrDoS_NTP        121368
TFTP              98917
Benign            97831
Syn               49373
UDP               18090
DrDoS_UDP         10420
UDP-lag            8872
MSSQL              8523
DrDoS_MSSQL        6212
DrDoS_DNS          3669
DrDoS_SNMP         2717
LDAP               1906
DrDoS_LDAP         1440
Portmap             685
NetBIOS             644
DrDoS_NetBIOS       598
UDPLag               55
WebDDoS              51
Name: Label, dtype: int64

as we see we put all the parquets into one parquet dataframe in order to facilitate the preprocessing .

In [8]:
#now let's convert from finaldataset.parquet into finaldataset.csv
csv_file = "/content/finaldataset.csv"
dfp.to_csv(csv_file, index=False)


In [9]:
#let's check our finaldataset.csv that we are going to use .
ddosdataset = pd.read_csv('/content/finaldataset.csv')
ddosdataset.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,17,48,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
1,17,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
2,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
3,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,1480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
4,17,1,2,0,2896.0,0.0,1448.0,1448.0,1448.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS


In [11]:
 ddosdataset['Label'].value_counts()

DrDoS_NTP        121368
TFTP              98917
Benign            97831
Syn               49373
UDP               18090
DrDoS_UDP         10420
UDP-lag            8872
MSSQL              8523
DrDoS_MSSQL        6212
DrDoS_DNS          3669
DrDoS_SNMP         2717
LDAP               1906
DrDoS_LDAP         1440
Portmap             685
NetBIOS             644
DrDoS_NetBIOS       598
UDPLag               55
WebDDoS              51
Name: Label, dtype: int64

now finaldataset is ready to be preprocessed .

**1-Handling the missing values in our dataset:**

---



to handle missing values in your dataset there are two methods:

1) dropping

2) imputation





In [12]:
# let's check if our dataset has any missing values .
missing_counts = ddosdataset.isnull().sum()
print(missing_counts.head(78))


Protocol                    0
Flow Duration               0
Total Fwd Packets           0
Total Backward Packets      0
Fwd Packets Length Total    0
                           ..
Idle Mean                   0
Idle Std                    0
Idle Max                    0
Idle Min                    0
Label                       0
Length: 78, dtype: int64


as we see our dataset has not any missing values.

**In case you found missing values:**

this is how you will deal with them --->

1) **imputation**:

you will analyse the data distribution in the coulmn who has missing values , then by analysing you will chose one of the central tendencies which are:

1-mean()

2-median()

3-mode()

**Exemple:**

this is how to analyse data distribution in specific coulmn which contains missing values(of course you need to import the needed libraries firstly):


fig ,ax = plt.subplots(figsize=(8,8))

sns.distplot(ddosdataset.columnnamewhichcontainsmissingvalues)

**sns---> seaborn library**

if data is gathered in one place , we will use Median to handle missing values.

**and this is how to do it:**

ddosdataset['columnwhichcontainsmissingvalues'].fillna(ddosdataset['columnwhichcontainsmissingvalues'].median(),inplace=True)

**Now if you check for missing values in this column you will find nothing**



2) **Dropping**:

here you will just dropp out all the missing values (**Note**:if your dataset is small then this method it's not a good solution )

**This is how to do it:**

**ddosdataset = ddosdataset.dropna(how='any')**

You will notice that the number of rows will be decreased.

**seperating data features and target(Label)**

In [13]:
X = ddosdataset.drop(columns='Label', axis =1)
Y = ddosdataset['Label']

In [14]:
print(X)

        Protocol  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0             17             48                  2                       0   
1             17              2                  2                       0   
2             17              1                  2                       0   
3             17              1                  2                       0   
4             17              1                  2                       0   
...          ...            ...                ...                     ...   
431366         6             53                  1                       2   
431367         6             66                  1                       2   
431368        17          26288                  2                       2   
431369         6          30679                  1                       1   
431370         6             67                  1                       2   

        Fwd Packets Length Total  Bwd Packets Length Total  \
0

In [15]:
print(Y)

0         DrDoS_DNS
1         DrDoS_DNS
2         DrDoS_DNS
3         DrDoS_DNS
4         DrDoS_DNS
            ...    
431366       Benign
431367       Benign
431368       Benign
431369       Benign
431370       Benign
Name: Label, Length: 431371, dtype: object


**So this is our Label that we are going to use in order to say it's a ddos attack or not**

Notice that this label contains non-numerical values which is not good for training our model that's why we should call label encoding concept.

**#### Label encoding is converting the labels into numeric form #####**.

**i will give the number 1 to all ddos attacks and number 0 to Benign**

**1------> DDOS Attack (dosn't matter it's type )**

0------> Benign (normal)

In [16]:
list(Y)

['DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDoS_DNS',
 'DrDo

In [17]:
#this loop will replace our label by numerical values ,if it's benign ,we will give 0 else we give it 1.
for i in range(len(Y)):
    if Y[i] == "Benign":
        Y[i] = 0
    else:
        Y[i] = 1

<ipython-input-17-50bb82cb8ca7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[i] = 1
<ipython-input-17-50bb82cb8ca7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[i] = 0


In [18]:
print(Y)

0         1
1         1
2         1
3         1
4         1
         ..
431366    0
431367    0
431368    0
431369    0
431370    0
Name: Label, Length: 431371, dtype: object


**Handling imbalanced data**

In [19]:
ddosdataset.tail()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
431366,6,53,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
431367,6,66,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
431368,17,26288,2,2,96.0,96.0,48.0,48.0,48.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
431369,6,30679,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
431370,6,67,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [20]:
benign = ddosdataset[ddosdataset.Label == 0]
ddos= ddosdataset[ddosdataset.Label ==1]

In [21]:
print(benign.shape)
print(ddos.shape)

(97831, 78)
(333540, 78)


In [22]:

ddos_sample = ddos.sample(n=97831)
print(ddos_sample.shape)


(97831, 78)


In [23]:
newddos_dataset = pd.concat([ddos_sample, benign] , axis= 0)

In [24]:
newddos_dataset.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
143223,17,503,26,0,10720.0,0.0,440.0,80.0,412.30768,97.828735,...,20,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
333434,17,3005933,4,0,2064.0,0.0,516.0,516.0,516.00000,0.000000,...,20,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
284365,17,3000937,4,0,2064.0,0.0,516.0,516.0,516.00000,0.000000,...,14,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
135883,17,448,38,0,16720.0,0.0,440.0,440.0,440.00000,0.000000,...,8,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
206762,6,44674785,6,4,36.0,24.0,6.0,6.0,6.00000,0.000000,...,20,87.0,19.79899,101.0,73.0,22337304.0,10303431.0,29622932.0,15051679.0,1


In [25]:
newddos_dataset['Label'].value_counts()

1    97831
0    97831
Name: Label, dtype: int64

**the data now is balanced**

saving this dataset

In [26]:
df2 = pd.DataFrame(newddos_dataset)
df2.to_csv('ddosdatasetwhitoutstd.csv', index=False)


**Data** **Standardization**:

**Data Standardization** is the process of standardizing the data to common format and common range.

In [27]:
dataframe = pd.read_csv('/content/ddosdatasetwhitoutstd.csv')

In [28]:
dataframe['Label'].value_counts()
dataframe.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,17,503,26,0,10720.0,0.0,440.0,80.0,412.30768,97.828735,...,20,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
1,17,3005933,4,0,2064.0,0.0,516.0,516.0,516.00000,0.000000,...,20,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
2,17,3000937,4,0,2064.0,0.0,516.0,516.0,516.00000,0.000000,...,14,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
3,17,448,38,0,16720.0,0.0,440.0,440.0,440.00000,0.000000,...,8,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1
4,6,44674785,6,4,36.0,24.0,6.0,6.0,6.00000,0.000000,...,20,87.0,19.79899,101.0,73.0,22337304.0,10303431.0,29622932.0,15051679.0,1


In [29]:
XF = dataframe.drop(columns='Label', axis =1)
YF = dataframe['Label']

In [30]:
scaler = StandardScaler()

In [31]:
standardized_data = scaler.fit_transform(XF)

In [32]:
print(standardized_data)

[[ 0.81538086 -0.37362543  0.11762244 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [ 0.81538086 -0.25675592 -0.21930163 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [ 0.81538086 -0.25695019 -0.21930163 ... -0.1959047  -0.31412701
  -0.2753953 ]
 ...
 [ 0.81538086 -0.37262275 -0.24993109 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [-1.18981521 -0.372452   -0.26524582 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [-1.18981521 -0.37364239 -0.26524582 ... -0.1959047  -0.31412701
  -0.2753953 ]]


In [33]:
XF = standardized_data

In [34]:
print(XF.shape)

(195662, 77)


In [35]:
print(YF)

0         1
1         1
2         1
3         1
4         1
         ..
195657    0
195658    0
195659    0
195660    0
195661    0
Name: Label, Length: 195662, dtype: int64


In [36]:
print(YF.shape)

(195662,)


***Now our Data is Ready to be splitted into trainning data and testing data ***

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(XF, YF, test_size=0.2, random_state=2)

In [38]:
print(X_train)

[[ 0.81538086 -0.37284188 -0.24993109 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [-1.18981521  2.54327685  0.20951082 ... 15.22628417  4.99861452
   1.02724383]
 [-1.18981521 -0.02328333 -0.08146906 ... -0.1959047  -0.31412701
  -0.2753953 ]
 ...
 [ 0.81538086 -0.25281387 -0.21930163 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [-1.18981521 -0.37363974 -0.23461636 ... -0.1959047  -0.31412701
  -0.2753953 ]
 [ 0.81538086 -0.37356329  2.78238557 ... -0.1959047  -0.31412701
  -0.2753953 ]]


In [39]:
print(X_train.shape, X_test.shape)

(156529, 77) (39133, 77)


In [40]:
print(Y_train.shape, Y_test.shape)

(156529,) (39133,)


let's turn our x_train and y_train into numpy arrays

In [41]:
x_l=X_train.tolist()

In [42]:
y_l=Y_train.tolist()

In [43]:
x_a=np.array(x_l)

In [44]:
y_a=np.array(y_l)

In [45]:
x_l=[]
y_l=[]

In [46]:
xt_l=X_test.tolist()

In [47]:
yt_l=Y_test.tolist()

In [48]:
xt_a=np.array(xt_l)

In [49]:
yt_a=np.array(yt_l)

In [50]:
xt_l=[]
yt_l=[]

***The Data now is ready to be used ***

**Let's Build the neural network model**

In [90]:
#importing needed libraries
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


**we will use RandomForest**

In [91]:
res1 = time.time()

rf = RandomForestClassifier()
rf.fit(x_a , y_a)
res2 = time.time()
print('RandomForest  took ',res2-res1,'seconds')

RandomForest  took  29.206974744796753 seconds


In [92]:
#validating with kfold method
# Define the number of folds (K)
k = 5

# Create a K-Fold cross-validator
kf = KFold(n_splits=k)

# Perform K-fold cross-validation
scores = cross_val_score(rf, x_a, y_a, cv=kf)

# Print the accuracy for each fold
for fold_idx, score in enumerate(scores):
    print(f"Fold {fold_idx + 1} accuracy: {score}")

# Compute the mean accuracy and standard deviation across all folds
mean_accuracy = np.mean(scores)

print(f"\nMean accuracy: {mean_accuracy}")

Fold 1 accuracy: 0.9990097744841244
Fold 2 accuracy: 0.9989458889669712
Fold 3 accuracy: 0.9991694882770076
Fold 4 accuracy: 0.9991694882770076
Fold 5 accuracy: 0.9989458552946814

Mean accuracy: 0.9990480990599584


**testing the model**

In [93]:
y_pred1 = rf.predict(xt_a)

#Calculate the accuracy
accuracy = accuracy_score(yt_a, y_pred1)
print("Accuracy score:", accuracy)

Accuracy score: 0.9991567219482278


**precision**

In [94]:
# Calculate the precision score
precision = precision_score(yt_a, y_pred1, average='weighted')
print("Precision score:", precision)

Precision score: 0.9991572929396988


**recall**

In [95]:
# Calculate recall
recall = recall_score(yt_a , y_pred1)

print("Recall:", recall)

Recall: 0.9986139630390144


**F1-score**

In [96]:
# Calculate the F1 score
f1 = f1_score(yt_a, y_pred1)

print("F1 Score: ", f1)

F1 Score:  0.9991525206091578


In [89]:
print(y_pred1)

[1 0 0 ... 0 1 0]


In [97]:
print(yt_a)

[1 0 0 ... 0 1 0]


**as we see the predicted values are like the real values , which means that our model works good with a high value of precision**